In [1]:
import numpy as np
from sklearn.cluster import KMeans
import mxnet as mx

/home/tapir/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
full_arr=np.random.rand(32,16,3,3)


In [ ]:
shape=full_arr.shape
shaped_arr=full_arr.reshape(shape[0]*shape[1],shape[2]*shape[3])

In [ ]:
estimator = KMeans(n_clusters=32)
estimator.fit(shaped_arr)

In [ ]:
indices = estimator.predict(X=shaped_arr)

In [ ]:
quantized_arr = np.array([estimator.cluster_centers_[idx] for idx in indices])

In [ ]:
quantized_arr=quantized_arr.reshape((512,3,3))

In [ ]:
cluster_centers, reconstructed_indices=np.unique(quantized_arr, return_inverse=True, axis=0)

In [ ]:
print indices
print reconstructed_indices

In [ ]:
quantized_reconstructed=np.take(cluster_centers,reconstructed_indices, axis=0)

In [ ]:
cluster_centers.shape

In [ ]:
quantized_reconstructed.shape

In [ ]:
np.array_equal(quantized_arr,quantized_reconstructed)

In [ ]:
import mxnet as mx
prefix="cnn_models/resnet20_clustered8"
epoch=0
sym, args, auxs = mx.mod.module.load_checkpoint(prefix, epoch)

In [ ]:
filter=args["stage2_unit2_conv2_weight"]

In [ ]:
shape=filter.shape
filter_indices=np.zeros((shape[0],shape[1]))
cluster_centers=np.zeros((shape[0],4,3,3))

In [ ]:
for ch in shape[1]:
    cluster_centers, reconstructed_indices=np.unique(filter[:,ch,:,:].asnumpy(), return_inverse=True, axis=0)
    

In [ ]:
cluster_centers.shape

In [ ]:
filter[:,0,:,:]

In [ ]:
a=np.array([[1,2,3],[4,5,6]])
a.shape

In [ ]:
a.reshape((1,6))

In [2]:
################# lookup forming flow ##################
img=mx.nd.random.uniform(0, 1, shape=(7,16,32,32))
convfilter=mx.nd.random.uniform(0, 1, shape=(4,16,3,3))

In [5]:
def convolve_codebook(data, codebook):
    filters = codebook
    fshape  = filters.shape
    
    filters = mx.nd.transpose(filters, axes=(1,0,2,3)).reshape((-1,1,0, 0)) 
    res = mx.nd.Convolution(data=data, weight=filters, num_group=fshape[1], num_filter=fshape[0]*fshape[1], no_bias=True, kernel=(3,3))
    res = res.expand_dims(1)
    res = res.reshape((0,fshape[1],fshape[0], 0, 0))
    res = mx.nd.transpose(res,axes=(0,2,1,3,4))
    
    return res

In [3]:
mx.nd.transpose(convfilter, axes=(1,0,2,3)).reshape((-1,1,0, 0)).shape

(64L, 1L, 3L, 3L)

In [18]:
#transpose+reshape the filters with shape: (n_clusters, in_channel, k, k) into (n_clusters*in_channel,1,k,k) for dw_conv
convfilter1=mx.nd.transpose(convfilter,axes=(1,0,2,3)).reshape((64,1,3,3)) 

In [19]:
res=mx.ndarray.Convolution(data=img, weight=convfilter1, num_group=16, num_filter=64, no_bias=True, kernel=(3,3))

In [20]:
res.shape

(7L, 64L, 30L, 30L)

In [21]:
res.expand_dims(1).shape

(7L, 1L, 64L, 30L, 30L)

In [6]:
#reshape to (in_channel, n_cluster, W, H) and transpose to (n_clusters, in_channel, W, H)
rshape=res.shape
res=res.reshape((16,4,rshape[2],rshape[3]))
res=mx.nd.transpose(res,axes=(1,0,2,3))
## end of flow

In [7]:
#convfilter2=convfilter.reshape((4,16,3,3))
convfilter2=convfilter[0,:,:,:]

In [8]:
convfilter2.shape

(16L, 3L, 3L)

In [9]:
res2=mx.ndarray.Convolution(data=img, weight=convfilter2.reshape((16,1,3,3)), num_group=16, num_filter=16, no_bias=True, kernel=(3,3))

In [10]:
print res[0].shape
print res2[0].shape

(16L, 30L, 30L)
(16L, 30L, 30L)


In [12]:
np.array_equal(res[1].asnumpy(),res2[0].asnumpy())

True

In [ ]:
res[0][0].shape

In [ ]:
a=mx.nd.array([[1,2,3,4],[5,6,7,8]])

In [ ]:
mx.nd.transpose(a,(1,0)).reshape((1,8))

In [ ]:
convfilter3=mx.nd.transpose(convfilter,axes=(1,0,2,3))

In [ ]:
res3=mx.ndarray.Convolution(data=img, weight=convfilter3, num_group=16, num_filter=64, no_bias=True)

In [4]:
#============================================

img=mx.nd.random.uniform(0, 1, shape=(7,16,32,32))
convfilter=mx.nd.random.uniform(0, 1, shape=(4,16,3,3))

In [6]:
res=convolve_codebook(img,convfilter)

In [7]:
res.shape

(7L, 4L, 16L, 30L, 30L)

In [10]:
convfilter2=convfilter[1,:,:,:]

In [9]:
convfilter2.shape

(16L, 3L, 3L)

In [11]:
res2=mx.ndarray.Convolution(data=img, weight=convfilter2.reshape((16,1,3,3)), num_group=16, num_filter=16, no_bias=True, kernel=(3,3))

In [12]:
res2.shape

(7L, 16L, 30L, 30L)

In [14]:
np.array_equal(res[0][1].asnumpy(),res2[0].asnumpy())

True